In [2]:
# Downloading necessary files
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import mplfinance as mpf 
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from datetime import datetime
import plotly.graph_objects as go
import re
import json
import requests 
import codecs
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.offline as pyo


# display plotly in browser when run in Spyder
# pio.renderers.default = 'browser'

In [3]:
# stock =  input('Type in the stock symbol: ').upper()
stock = 'LXRX'

In [4]:
def create_df():

    # =============================================================================
    # Creating a new dataset
    # =============================================================================
    
    global df
    
    start = "2009-01-01"
    end = dt.datetime.now()
    short_sma = 50
    long_sma = 200
    
    # creating a dataset for selected stock 
    df = yf.download(stock, start, end,interval = '1d')
    df = pd.DataFrame(df.dropna(), columns = ['Open', 'High','Low','Close', 'Adj Close', 'Volume'])


    # preparing data from time series analysis
    # eliminating any NAs 
    df.index = pd.to_datetime(df.index)
    df = df.asfreq('B')

    print('============================================================')
    print('Nan in each columns before Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
    
    df = df.bfill()
    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('The df has {} rows and {} columns.'.format(*df.shape))
#    df = df.dropna()
    
    # Create short SMA
    df['short_SMA'] = df.iloc[:,1].rolling(window = short_sma).mean()
    
    # Create Long SMA
    df['long_SMA'] = df.iloc[:,1].rolling(window = long_sma).mean()
    
    # Create daily_return
    df['daily_return'] = df['Close'].pct_change(periods=1).mul(100)
    
    # Create monthly_return
    df['monthly_return'] = df['Close'].pct_change(periods=30).mul(100)
    
    # Create annual_return
    df['annual_return'] = df['Close'].pct_change(periods=365).mul(100)
    df['Name'] = stock

    print('============================================================')
    print(stock, '- Full Dataset')
    print('------------------------------------------------------------')
    print(df.head())
    print('------------------------------------------------------------')
    print(stock, 'Full Dataset - Column Names')
    print(df.columns)

    print('============================================================')
    print('Nan in each columns after Backfill for' , stock)
    print('------------------------------------------------------------')  
    print(df.isna().sum())
    print('============================================================')
    
#     # write to csv
#     df.to_csv(r"df.csv", index =  True)
    

In [5]:
create_df()

[*********************100%***********************]  1 of 1 completed
Nan in each columns before Backfill for LXRX
------------------------------------------------------------
Open         117
High         117
Low          117
Close        117
Adj Close    117
Volume       117
dtype: int64
The df has 3369 rows and 6 columns.
Nan in each columns after Backfill for LXRX
------------------------------------------------------------
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
The df has 3369 rows and 6 columns.
LXRX - Full Dataset
------------------------------------------------------------
             Open   High    Low  Close  Adj Close   Volume  short_SMA  \
Date                                                                    
2009-01-02   9.80  10.64   9.03  10.08      10.08  26943.0        NaN   
2009-01-05  10.15  10.43   8.82  10.29      10.29  16286.0        NaN   
2009-01-06  10.36  11.13  10.29  11.06      11.06  11800.

In [6]:
def create_y():
    
    global y
    
    y = pd.DataFrame(df['Close'], columns = ['Close'])
    y.sort_index(inplace = True)
    # dropping the first na (because there is no difference)
    y = y.dropna()
    y['Name'] = stock

    # Create SMA 16-Period for MACD
    y['SMA_12'] = df.iloc[:,1].rolling(window = 12).mean()
    
    # Create SMA 16-Period for MACD
    y['SMA_26'] = df.iloc[:,1].rolling(window = 26).mean()
    
    y['MACD'] = y['SMA_12'] -  y['SMA_26']
    
    # examining the pct_change
    y['Close Percentage Change'] = y['Close'].pct_change(1)
    
    # Creating a new variable, examining the difference for each observation
    y['diff'] = y['Close'].diff()

    # logging the target varialbe due to great variance
    y['log_Close'] = np.log(y['Close'])
    
    # Creating a new variable, examining the difference for each observation
    y['log_Close_diff'] = y['log_Close'].diff()
    
    y['Logged Close Percentage Change'] = y['log_Close'].pct_change(1)
    
    # dropping the first na (because there is no difference)
    y = y.dropna()
    

#    # write to csv
#    y.to_csv(r"y.csv", index =  True)

    print('============================================================')
    print(stock, '- Target Variable')
    print('------------------------------------------------------------')
    print(y.head())
    print('------------------------------------------------------------')
    print('Column Names')
    print(y.columns)
    print('============================================================')

#     # write to csv
#     y.to_csv(r"y.csv", index =  True)
    


In [7]:
create_y()

LXRX - Target Variable
------------------------------------------------------------
            Close  Name     SMA_12     SMA_26      MACD  \
Date                                                      
2009-02-06  11.55  LXRX   9.823334  10.098846 -0.275513   
2009-02-09  11.90  LXRX  10.085833  10.147308 -0.061474   
2009-02-10  10.78  LXRX  10.377500  10.217308  0.160192   
2009-02-11  10.15  LXRX  10.535000  10.211923  0.323077   
2009-02-12  10.43  LXRX  10.663333  10.201154  0.462179   

            Close Percentage Change      diff  log_Close  log_Close_diff  \
Date                                                                       
2009-02-06                 0.078431  0.840000   2.446685        0.075508   
2009-02-09                 0.030303  0.349999   2.476538        0.029853   
2009-02-10                -0.094118 -1.120000   2.377693       -0.098846   
2009-02-11                -0.058442 -0.630000   2.317474       -0.060219   
2009-02-12                 0.027586  0.280001 

In [8]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,short_SMA,long_SMA,daily_return,monthly_return,annual_return,Name
Date,,,,,,,,,,,,
2021-11-25,5.00,5.03,4.720,4.740,4.740,518400.0,5.3228,5.11685,-7.782103,-23.051950,144.329878,LXRX
2021-11-26,5.00,5.03,4.720,4.740,4.740,518400.0,5.3262,5.10275,0.000000,-23.051950,148.167532,LXRX
2021-11-29,4.83,4.86,4.630,4.660,4.660,693200.0,5.3284,5.08800,-1.687762,-14.338239,143.979054,LXRX
2021-11-30,4.59,4.68,4.380,4.580,4.580,753700.0,5.3258,5.07380,-1.716737,-8.946327,141.052631,LXRX
2021-12-01,4.60,4.87,4.585,4.775,4.775,285020.0,5.3280,5.05745,4.257646,-6.372545,147.409338,LXRX


In [11]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 


trace =  go.Candlestick(
                x = df.index,
                open = df["Open"],
                high = df["High"],
                low = df["Low"],
                close = df["Close"],
                name = stock)
short_sma = go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="blue"))

long_sma = go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="black"))

data = [trace, short_sma, long_sma]
layout = dict(
    title = 'Price of {}'.format(stock),
    yaxis_title = 'US Dollars',
    yaxis_tickprefix = '$', yaxis_tickformat = ',.2f',
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
# fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.5,
    xanchor="right",
    x=1
))

fig.update_xaxes(
    rangeslider_visible=True,
    tickformatstops = [
        dict(dtickrange=[604800000, "M6"], value="%e. %b w"),
        dict(dtickrange=["M6", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ],
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
    ]
)


def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 0.25, in_view.High.max() + 0.25]

fig.layout.on_change(zoom, 'xaxis.range')
fig

FigureWidget({
    'data': [{'close': array([10.07999992, 10.28999996, 11.06000042, ...,  4.65999985,  4.57999…

In [10]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 

trace = go.Bar(x = df.index,
                    y = df['Volume'],
                    name = 'Volume',
                    customdata = df['Name'],
                    hovertemplate="<b>%{customdata}</b><br><br>" +
                                    "Date: %{x|%d %b %Y} <br>" +
                                    "Volume: %{y:,.}<br>" +
                                    "<extra></extra>",
                    marker = dict(color="black"))

data = [trace]
layout = dict(
    title = 'Volume of {} (zoom in)'.format(stock),
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
fig.update_layout(showlegend=False)

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=False)


fig.update_xaxes(
    rangeslider_visible=True,
    tickformatstops = [
        dict(dtickrange=["M1", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ],
        rangebreaks=[
        dict(bounds=["sat", "mon"])], #hide weekends
)



def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Volume.min() - 0.25, in_view.Volume.max() + 500000]

fig.layout.on_change(zoom, 'xaxis.range')
fig

FigureWidget({
    'data': [{'customdata': array(['LXRX', 'LXRX', 'LXRX', ..., 'LXRX', 'LXRX', 'LXRX'], dtype=…